In [ ]:
# 크롤링 모듈 import
from wevity_crawler import crawl_wevity
from dacon_crawler import crawl_dacon
from devEvent_crawler import crawl_devEvent
from inflearn_crawler import crawl_inflearn

# 각 사이트에서 데이터 크롤링
df_wevity = crawl_wevity()
df_dev = crawl_devEvent()
df_inflearn = crawl_inflearn()
df_dacon = crawl_dacon(10)

In [ ]:
import pandas as pd

# 데이터 통합
df = pd.concat([df_wevity, df_dacon, df_dev, df_inflearn], ignore_index=True)
print(f"총 {len(df)}개 이벤트 통합됨")

In [ ]:
# 정규표현식을 이용한 데이터 전처리
import re
from datetime import datetime
import pandas as pd

def normalize_date(date_str: str) -> str:
    """
    날짜 문자열을 'YYYY.MM.DD ~ YYYY.MM.DD' 형식으로 통일한다.

    규칙
    ───────────────────────────────────────────────
    1. 괄호 안 정보·시간(HH:MM, HHMM)·불필요 공백 제거
    2. 'YYYY.MM.DD ~ YYYY.MM.DD'  → 그대로 포맷
    3. 'YYYY.MM.DD ~ MM.DD'       → 뒤쪽 연도 = 앞쪽 연도
    4. 'YYYY.MM.DD … ~ HH:MM'     → 같은 날 범위
    5. 단일 날짜(YYYY.MM.DD)      → 앞뒤 날짜 동일
    """
    date_str = str(date_str).strip()

    # ── (1) 노이즈 제거 ──────────────────────────────────
    date_str = re.sub(r'\(.*?\)', '', date_str)               # (요일) (한국 표준시)
    date_str = re.sub(r'\b\d{1,2}:\d{2}\b', '', date_str)     # 09:00
    # 4자리 시간: 공백/틈 뒤 & 공백/문자열끝 앞에 위치할 때만 제거
    date_str = re.sub(r'(?<=\s)(?:[01]\d[0-5]\d|2[0-3][0-5]\d)(?=\s|$)', '', date_str)
    date_str = date_str.replace('-', '.').replace('/', '.')
    date_str = re.sub(r'\s+', ' ', date_str).strip()

    # ── (2) YYYY.MM.DD ~ YYYY.MM.DD ─────────────────────
    m = re.search(
        r'(\d{4})\.(\d{1,2})\.(\d{1,2})\s*~\s*'
        r'(\d{4})\.(\d{1,2})\.(\d{1,2})', date_str)
    if m:
        y1, m1, d1, y2, m2, d2 = map(int, m.groups())
        return f"{datetime(y1, m1, d1):%Y.%m.%d} ~ {datetime(y2, m2, d2):%Y.%m.%d}"

    # ── (3) YYYY.MM.DD ~ MM.DD  (뒤 연도 생략) ───────────
    m = re.search(
        r'(\d{4})\.(\d{1,2})\.(\d{1,2})\s*~\s*'
        r'(\d{1,2})\.(\d{1,2})', date_str)
    if m:
        y, m1, d1, m2, d2 = map(int, m.groups())
        start = datetime(y,  m1, d1)
        end   = datetime(y,  m2, d2)
        if end < start:                       # 연말 넘기면 +1y
            end = datetime(y+1, m2, d2)
        return f"{start:%Y.%m.%d} ~ {end:%Y.%m.%d}"

    # ── (4) YYYY.MM.DD … ~ HH:MM (같은 날) ──────────────
    m = re.search(r'(\d{4})\.(\d{1,2})\.(\d{1,2})\s*~', date_str)
    if m:
        y, mth, d = map(int, m.groups())
        dt = datetime(y, mth, d)
        return f"{dt:%Y.%m.%d} ~ {dt:%Y.%m.%d}"

    # ── (5) 단일 날짜 (YYYY.MM.DD) ──────────────────────
    m = re.match(r'(\d{4})\.(\d{1,2})\.(\d{1,2})$', date_str)
    if m:
        y, mth, d = map(int, m.groups())
        dt = datetime(y, mth, d)
        return f"{dt:%Y.%m.%d} ~ {dt:%Y.%m.%d}"

    # 이외 패턴은 원본 유지
    return date_str

# date 컬럼을 'YYYY.MM.DD ~ YYYY.MM.DD' 형식으로 통일
df["date"] = df["date"].apply(normalize_date)

In [16]:
# LLM을 이용한 데이터 전처리 
import os, time, textwrap, json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv

# 환경변수 로드 및 OpenAI 클라이언트 설정
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def get_description(url: str, *, retry: int = 3) -> str:
    """웹프리뷰로 URL 핵심 내용(설명) 가져오기"""
    prompt = f"{url}\n\n이 페이지의 주요 정보를 7줄 이내의 줄글 형식으로 문단형 요약을 작성해 주세요."
    for _ in range(retry):
        try:
            res = client.responses.create(
                model="gpt-4.1",
                tools=[{"type": "web_search_preview"}],
                input=prompt,
            )
            return res.output_text.strip()
        except Exception as e:
            print("⚠️ desc 재시도:", e)
            time.sleep(2)
    return ""

def classify(text: str) -> tuple[bool, str]:
    """제목+설명 텍스트 → (is_dev_event, category)"""
    system = "당신은 행사 정보를 분석해 JSON으로 반환하는 AI입니다."
    user = f"""
──── 행사 정보 ────
{text}
──────────────────
1) 개발자·기술 행사면 "yes", 아니면 "no"
2) 카테고리는 하나 선택:
   공모전/대회 | 부트캠프/교육 | 컨퍼런스/포럼 | 밋업/네트워킹 | 기타
JSON만:
{{"is_dev_event":"yes","category":"부트캠프/교육"}}
"""
    res = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "system", "content": system},
                  {"role": "user",   "content": textwrap.dedent(user)}],
        response_format={"type": "json_object"},
        temperature=0,
        max_tokens=120,
    )
    try:
        data = json.loads(res.choices[0].message.content)
    except Exception as e:
        print("⚠️ JSON 파싱 실패:", e)
        return False, "기타"

    is_dev = str(data.get("is_dev_event", "")).lower().startswith("y")
    cat    = data.get("category", "기타")
    if cat not in ['공모전/대회','부트캠프/교육','컨퍼런스/포럼','밋업/네트워킹','기타']:
        cat = "기타"
    return is_dev, cat

# ─── 실행 셀 (DataFrame 처리) ────────────────────────────
tqdm.pandas()

def pipeline(row):
    desc = get_description(row["url"])
    merged = f"제목: {row['title']}\n설명: {desc}"
    is_dev, cat = classify(merged)
    return pd.Series({"description": desc,
                      "is_dev_event": is_dev,
                      "category": cat})

df[["description", "is_dev_event", "category"]] = (
    df.progress_apply(pipeline, axis=1)
)

df[["title", "description", "is_dev_event", "category"]].to_csv("data_with_desc_dev_category.csv", index=False, encoding="utf-8-sig")
df.to_csv("data_with_new_features.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료 → data_with_new_features.csv")

100%|██████████| 75/75 [07:45<00:00,  6.21s/it]

✅ 저장 완료 → data_with_desc_dev_category.csv


In [20]:
df.head()

,title,host,date,url,description,is_dev_event,category


In [19]:
# 개발 행사 분류
# TRUE인 행만 필터링
df = df[df["is_dev_event"] == "TRUE"].copy()

# 컬럼 삭제
df.drop(columns=["is_dev_event"], inplace=True)

# 저장
df.to_csv("preprocessed_data.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료 → preprocessed_data.csv")


✅ 저장 완료 → preprocessed_data.csv


In [14]:
!pip install sentence-transformers

  Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 50.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.8 MB/s eta 0:00:00
Using cached fsspec-2025.5.1-py3-none-any.whl (199 kB)
Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl (284 kB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)


In [ ]:
# 문장 유사도 계산
from sentence_transformers import SentenceTransformer, util

# 1. 모델 로드 (최초 1회만 필요)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [15]:
# 2. 사용자 관심 키워드 → 문장형으로 변환
user_keywords = ["백엔드", "클라우드", "AI", "LLM"]
user_sentence = f"이 행사는 {', '.join(user_keywords)} 기술에 관심 있는 개발자를 위한 프로그램입니다."

# 3. 비교 대상 설명 (예: 이벤트 설명 요약)
event_description = """
AWS TechCamp는 AWS 클라우드 기초부터 심화까지 배우는 3일 과정의 무료 교육 프로그램입니다.
이론과 실습을 병행하며, 온라인과 오프라인으로 운영됩니다.
세션은 수준별·산업별로 다양하게 구성되어 있습니다.
클라우드 입문자부터 실무자까지 모두 참여할 수 있습니다.
"""

# 4. 벡터 임베딩
emb_user = model.encode(user_sentence, convert_to_tensor=True)
emb_event = model.encode(event_description, convert_to_tensor=True)

# 5. 코사인 유사도 계산
similarity = util.cos_sim(emb_user, emb_event).item()

# 6. 결과 출력
print(f"📊 유사도 점수: {similarity:.4f}")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔍 사용자 관심 문장: 이 행사는 백엔드, 클라우드, AI, LLM 기술에 관심 있는 개발자를 위한 프로그램입니다.
📝 이벤트 설명 요약: AWS TechCamp는 AWS 클라우드 기초부터 심화까지 배우는 3일 과정의 무료 교육 프로그램입니다.
이론과 실습을 병행하며, 온라인과 오프라인으로 운영됩니다.
세션은 수준별·산업별로 다양하게 구성되어 있습니다.
클라우드 입문자부터 실무자까지 모두 참여할 수 있습니다.
📊 유사도 점수: 0.6317
